# Project Zeus
## Created by Michael Wentz

### Import Modules

In [1]:
import numpy as np
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual, VBox, FloatSlider
import ipywidgets as widgets

### Load production data from excel file

In [2]:
file = 'https://raw.githubusercontent.com/mwentzWW/petrolpy/master/Zeus/Sample_Production/Sample_Prod_Data.xlsx'
well_data = pd.read_excel(file)
well_data.columns = [c.lower() for c in well_data.columns]
well_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704 entries, 0 to 703
Data columns (total 5 columns):
api number         704 non-null object
production date    704 non-null datetime64[ns]
oil_bbl            694 non-null float64
gas_mcf            694 non-null float64
water_bbl          693 non-null float64
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 27.6+ KB


### Check Data Layout

In [3]:
well_data.head()

,api number,production date,oil_bbl,gas_mcf,water_bbl
0,40 063 05037,1959-05-31,3533.0,0.0,0.0
1,40 063 05037,1959-06-30,4763.0,0.0,0.0
2,40 063 05037,1959-07-31,4475.0,1570.0,0.0
3,40 063 05037,1959-08-31,2620.0,917.0,0.0
4,40 063 05037,1959-09-30,3531.0,1236.0,0.0


### Delete API because there is only one well, and we don't need water

In [4]:
well_data.drop(columns=['api number', 'water_bbl'], inplace=True)
well_data.head()

,production date,oil_bbl,gas_mcf
0,1959-05-31,3533.0,0.0
1,1959-06-30,4763.0,0.0
2,1959-07-31,4475.0,1570.0
3,1959-08-31,2620.0,917.0
4,1959-09-30,3531.0,1236.0


### Replace NaN with zeros for computations

In [5]:
well_data.fillna(value=0, inplace=True)

### Calculate effective decline rate for oil and gas

In [6]:
qi_oil = well_data.oil_bbl[0:30].max()
qi_gas = well_data.gas_mcf[0:30].max()

In [7]:
well_data['eff_decline_oil'] = -(well_data.oil_bbl - qi_oil)/qi_oil
well_data['eff_decline_gas'] = -(well_data.gas_mcf - qi_gas)/qi_gas

In [8]:
well_data.head()

,production date,oil_bbl,gas_mcf,eff_decline_oil,eff_decline_gas
0,1959-05-31,3533.0,0.0,0.258241,1.000000
1,1959-06-30,4763.0,0.0,-0.000000,1.000000
2,1959-07-31,4475.0,1570.0,0.060466,-0.000000
3,1959-08-31,2620.0,917.0,0.449927,0.415924
4,1959-09-30,3531.0,1236.0,0.258661,0.212739


### Normalize data by time

In [9]:
num_days_online = len(well_data['oil_bbl'])
num_days_online

704

In [10]:
days = []
for day in range(0, num_days_online):
    days.append(day)
days[0:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [11]:
well_data['time'] = days
well_data.head()

,production date,oil_bbl,gas_mcf,eff_decline_oil,eff_decline_gas,time
0,1959-05-31,3533.0,0.0,0.258241,1.000000,0
1,1959-06-30,4763.0,0.0,-0.000000,1.000000,1
2,1959-07-31,4475.0,1570.0,0.060466,-0.000000,2
3,1959-08-31,2620.0,917.0,0.449927,0.415924,3
4,1959-09-30,3531.0,1236.0,0.258661,0.212739,4


### Calculate the type curve for oil and gas, assume b = 0.5 (will adjust later), use first 30 day average decline for di

In [12]:
well_data.fillna(value=0, inplace=True)
well_data.loc[well_data.eff_decline_oil > 10000] = 0
well_data.loc[well_data.eff_decline_gas > 10000] = 0

In [13]:
well_data.head()

,production date,oil_bbl,gas_mcf,eff_decline_oil,eff_decline_gas,time
0,1959-05-31 00:00:00,3533.0,0.0,0.258241,1.000000,0
1,1959-06-30 00:00:00,4763.0,0.0,-0.000000,1.000000,1
2,1959-07-31 00:00:00,4475.0,1570.0,0.060466,-0.000000,2
3,1959-08-31 00:00:00,2620.0,917.0,0.449927,0.415924,3
4,1959-09-30 00:00:00,3531.0,1236.0,0.258661,0.212739,4


In [14]:
def plot_oil(b_value=0.5, initial_decline_rate=0.15):
    well_data['type_curve_oil'] = qi_oil*((1 + b_value*initial_decline_rate*well_data.time)**(-1/b_value))
    well_data.type_curve_oil.astype('int')
    well_data.fillna(value=0, inplace=True)
    well_data.loc[well_data.type_curve_oil > qi_oil] = qi_oil

    fig, axes = plt.subplots( figsize=(12, 6))

    axes.plot(well_data.time, well_data.type_curve_oil, 'k--', lw =2, label='Type Curve')
    axes.plot(well_data.time, well_data.oil_bbl, 'r', label='Historical Production')
    axes.legend(loc=1);
    axes.grid(True)
    axes.set_xlabel('Time (Days)')
    axes.set_ylabel('Oil Rate (BPD)')
    axes.set_title('Oil Type Curve');

def plot_gas(b_value=0.5, initial_decline_rate=0.15):
    well_data['type_curve_gas'] = qi_gas*((1 + b_value*initial_decline_rate*well_data.time)**(-1/b_value))
    well_data.type_curve_gas.astype('int')
    well_data.fillna(value=0, inplace=True)
    well_data.loc[well_data.type_curve_gas > qi_gas] = qi_gas
    
    fig, axes = plt.subplots(figsize=(12, 6))

    axes.plot(well_data.time, well_data.type_curve_gas, 'k--', lw=2, label='Type Curve')
    axes.plot(well_data.time, well_data.gas_mcf, 'g', label='Historical Production')
    axes.legend(loc=1);
    axes.grid(True)
    axes.set_xlabel('Time (Days)')
    axes.set_ylabel('Gas Rate (MCFD)')
    axes.set_title('Oil Type Curve');

### Plot production with type curves

In [15]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
interact_oil = interact(plot_oil, b_value= (0, 1, 0.01), initial_decline_rate= (0, 1, 0.01))

interactive(children=(FloatSlider(value=0.5, description='b_value', max=1.0, step=0.01), FloatSlider(value=0.1…

## To-do

* add multiline segment functionality
* add optimized solution by minimizing the error